In [1]:
"""

1. Make main indexing dataset containing:
    - Longitude
    - Latitude
    - ISMIP grid cell index
    - Region
    - Elevation

"""

# Import libraries
import glob
import os
from datetime import datetime
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4
import pyresample
from pyproj import Transformer
import matplotlib.pyplot as plt

# Define base path
path = '/Users/jryan4/Dropbox (University of Oregon)/research/snowfall/'

# Read ISMIP data
ismip = xr.open_dataset(path + 'data/masks/1km-ISMIP6.nc')

# Define maximum snowline
snowline_file = netCDF4.Dataset(path + 'data/masks/monthly_bare_ice_2012.nc')
snowline = snowline_file.variables['bare_ice'][1, :, :].filled(np.nan)
max_snowline = (snowline > 0.1)

# Define regions
regions = xr.open_dataset('/Users/jryan4/Dropbox (University of Oregon)/research/clouds/data/temp_albedo_summer_climatologies.nc')

# Define ablation zone coordinates, elevations, and regions
abl_lon = ismip['lon'].values[max_snowline]
abl_lat = ismip['lat'].values[max_snowline]
abl_ele = ismip['SRF'].values[max_snowline]
abl_reg = regions['regions'].values[max_snowline]

/opt/anaconda3/envs/clouds/lib/python3.8/site-packages/pyresample/bilinear/__init__.py:50: UserWarning: XArray and/or zarr not found, XArrayBilinearResampler won't be available.
  warnings.warn("XArray and/or zarr not found, XArrayBilinearResampler won't be available.")


In [2]:
# Indices of ablation zone
idx, idy = np.where(max_snowline)

# Save main indexing dataset
ds_main = xr.Dataset(
    data_vars={
        "region": (("x"), np.array(abl_reg).astype('int8')),
        "elevation": (("x"), np.array(abl_ele).astype('float64')),
        "index_x": (("x"), np.array(idx).astype('int16')),
        "index_y": (("x"), np.array(idy).astype('int16')),
    },
    
    coords={
        "longitude": (('x',), np.array(abl_lon)),
        "latitude": (('x',), np.array(abl_lat)), 
    },
    attrs={
        "Produced": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "Author":'Johnny Ryan', 
        "Email":'jryan4@uoregon.edu'
    },
)

ds_main.to_netcdf(path + 'data/masks/index_main.nc')